In [7]:
import modern_robotics as mr
import numpy as np 
import csv
import math

In [8]:
L = .47/2
W = .3/2
R = .0475
F = (R/4)*np.array([[-1/(L + W), 1/(L + W), 1/(L + W), -1/(L + W)],
              [1, 1, 1, 1],
              [-1, 1, -1, 1]])

In [9]:
def to_csv(filename, res):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for r in res:
            writer.writerow(r)
            
def cap(x, x_cap):
    if x > x_cap:
        return float(x_cap)
    if x < -x_cap:
        return float(-x_cap)
    return float(x)

In [10]:
def NextState(config, ctrl, dt, rom):
    global F
    for i in range(len(ctrl)):
        # print(ctrl[i], rom[i])
        ctrl[i] = cap(ctrl[i], rom[i])
    # ctrl = np.nan_to_num(ctrl)
    
    wheel_vel = ctrl[:4]
    Vb = F @ wheel_vel * dt # (3, 4) * (4, 1)   
    wbz = Vb[0]
    if math.isclose(wbz, 0, abs_tol=.0001): 
        dphib = 0
        dxb = Vb[1]
        dyb = Vb[2]
    else:
        dphib = Vb[0]
        dxb = (Vb[1]*np.sin(Vb[0]) + Vb[2]*(np.cos(Vb[0]) - 1))/Vb[0]
        dyb = (Vb[2]*np.sin(Vb[0]) + Vb[1]*(np.cos(Vb[0]) - 1))/Vb[0]
    dqb = np.array([dphib, dxb, dyb])
    dq = np.array([[1, 0, 0],
                [0, np.cos(config[0]), -np.sin(config[0])],
                [0, np.sin(config[0]), np.cos(config[0])]]) @ dqb

    # update 
    new_joint = config[3:8] + ctrl[4:]*dt
    new_wheel = config[8:]+ ctrl[:4]*dt
    new_chassis = config[:3] + dq
    
    return np.array([*new_chassis, *new_joint, *new_wheel, 1])

In [11]:
config = np.zeros(12)
ctrl = np.array([-10., 10., -10., 10., 0., 0., 0., 0., 0.])
dt = .01
rom = np.ones(9)*100
rom, ctrl

(array([100., 100., 100., 100., 100., 100., 100., 100., 100.]),
 array([-10.,  10., -10.,  10.,   0.,   0.,   0.,   0.,   0.]))

In [12]:
res = []
for _ in range(100):
    config = NextState(config, ctrl, dt, rom)[:-1]
    res.append(config)
print(res[-1])

[  0.      0.      0.475   0.      0.      0.      0.      0.    -10.
  10.    -10.     10.   ]
